In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# K is for latent dim
K = 256
num_latent = 12

class Net(nn.Module):
    def __init__(self, K=K, num_latent=num_latent):
        super(Net, self).__init__()
        
        self.x_dim = x_dim
        self.h_dim_1 = 256
        self.h_dim_2 = 256
        self.z_dim = z_dim
        self.nc = nc
        self.encoder = nn.Sequential(
            nn.Linear(self.x_dim, self.h_dim_1, bias=True),
            nn.ReLU(True),
            nn.Linear(self.h_dim_1, self.h_dim_2, bias=True),
            nn.ReLU(True),
            nn.Linear(self.h_dim_2, self.z_dim * 2, bias=True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.z_dim, self.h_dim_2),
            nn.ReLU(True),
            nn.Linear(self.h_dim_2, self.h_dim_1, bias=True),
            nn.ReLU(True),
            nn.Linear(self.h_dim_1, self.x_dim, bias=True),
        )
        self.weight_init()
        
        self.fc1 = nn.Linear(784, 1024)
        self.fc2 = nn.Linear(1024,1024)
        self.enc_mean = nn.Linear(1024, K)
        self.enc_std = nn.Linear(1024, K)
        self.dec = nn.Linear(K, 10)
        self.num_latent = num_latent
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.constant_(self.fc2.bias, 0.0)
        nn.init.xavier_uniform_(self.enc_mean.weight)
        nn.init.constant_(self.enc_mean.bias, 0.0)
        nn.init.xavier_uniform_(self.enc_std.weight)
        nn.init.constant_(self.enc_std.bias, 0.0)
        nn.init.xavier_uniform_(self.dec.weight)
        nn.init.constant_(self.dec.bias, 0.0)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = 2*x - 1. # input normalize to [-1,1]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        enc_mean, enc_std = self.enc_mean(x), F.softplus(self.enc_std(x)-5)
        latent = enc_mean.unsqueeze(1) + torch.randn((tuple(enc_std.shape)[0],)\
                                        +(self.num_latent,)\
                                        + tuple(enc_std.shape)[1:]).cuda() * enc_std.unsqueeze(1)
        outputs = F.softmax(self.dec(latent),dim=2)
        output = torch.mean(outputs, dim=1)
        return outputs, output, enc_mean, enc_std
    
def update_target(target, original, update_rate):
        for target_param, param in zip(target.parameters(), original.parameters()):
            target_param.data.copy_((1.0 - update_rate) * target_param.data + update_rate*param.data)